In [1]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 18.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
# **IMPORTANT**: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from sklearn.preprocessing import MinMaxScaler
import warnings

import boto3
import s3fs
import random
from config_module import Config
config = Config()

# Set seed for reproducibility
np.random.seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import math
import pickle
from datetime import datetime, timedelta

import ast
import seaborn as sn
warnings.filterwarnings('ignore')
random.seed(12)

In [3]:
fs = s3fs.S3FileSystem()
bucket = config.get_value('bucket')
# prefix = config.get_value('prefix')
prefix_inp = config.get_value('prefix_inp')
#prefix_out = config.get_value('prefix_out')
inp_path = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}/"

In [4]:
start_time = datetime.now()

In [5]:
start_time

datetime.datetime(2024, 10, 22, 9, 20, 36, 599722)

In [6]:
# part_master_path_1 = f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/test_data/TPCAP Part Master New.xlsx"
# data_gt_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/GT_part_master.csv"
# data_pic_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/PIC_master.csv"
# data_supp_master_path_1=f"s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/to-be-removed/master_data/job3/etljob3input/Supplier_master.csv"

In [7]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [8]:
def get_latest_s3_file(bucket_name, prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        #print(obj)
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [9]:
inp_path

'persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/model-training-output/'

In [10]:
#prefix_output_file = config.get_value('prefix_output_file')
prefix_output = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"

bestfit_file_prophet = os.path.basename(get_latest_s3_file(bucket, inp_path,'train_prophet_forecast_'))
bestfit_file_lstm = os.path.basename(get_latest_s3_file(bucket, inp_path,'train_lstm_forecast_'))
bestfit_file_ridge = os.path.basename(get_latest_s3_file(bucket, inp_path,'train_ridge_forecast_'))
bestfit_file_lasso = os.path.basename(get_latest_s3_file(bucket, inp_path,'train_lasso_forecast_'))
bestfit_file_xgboost = os.path.basename(get_latest_s3_file(bucket, inp_path,'train_xgboost_forecast_'))
prophet_config_filename = config.get_value('prophet_config_filename')

In [11]:
print(bestfit_file_prophet)
print(bestfit_file_lstm)
print(bestfit_file_lasso)
print(bestfit_file_ridge)
print(bestfit_file_xgboost)

train_prophet_forecast_2024-09-13.xlsx
train_lstm_forecast_1_2024-09-13.xlsx
train_lasso_forecast_2024-09-13.xlsx
train_ridge_forecast_2024-09-13.xlsx
train_xgboost_forecast_2024-09-13.xlsx


In [12]:
training_date = pd.to_datetime(config.get_value('model_training_date'))
file_date_training = training_date.strftime("%Y-%m-%d")

In [13]:
def calculate_mape(x, y):
    """
    Function to calculate the Mean Absolute Percentage Error (MAPE) between two values x and y.
    
    Parameters:
    x: The true value.
    y: The predicted value.
    
    Returns:
    The MAPE value as a float.
    """
    
    # Check if x is NaN
    if math.isnan(x):
        return 0
    
    # Check if x and y are equal
    if x == y:
        return 1
    
    # Check if x is zero
    elif x == 0:
        return 0
    
    # Calculate MAPE
    else:
        mape = 1 - (abs(x - y) / x)
        return mape

In [14]:
def get_consecutive_months(start_date, n_months=3):
    """
    Generate a list of consecutive month start dates.

    Args:
    - start_date (str): Starting date in 'YYYY-MM-DD' format.
    - n_months (int): Number of consecutive months to generate.

    Returns:
    - list: List of consecutive month start dates in 'YYYY-MM-DD' format.
    """
    start_date = pd.to_datetime(start_date)
    month_list = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, n_months + 1)]
    return month_list

In [15]:
# def prophet_best_fit_lag1(bucket_name, prophet_training_prefix, prophet_filename, model_config_filename, training_end_date):
def prophet_best_fit_lag1(bucket_name, prophet_training_prefix, prophet_filename, training_end_date):

    """
    Combine Prophet forecasts from three different lags into a single formatted dataframe for lag 1.

    Args:
    - prophet_lag1_url (str): File path or URL to Prophet forecasts
    - training_end_date: end date for training data

    Returns:
    - DataFrame: Combined and formatted dataframe containing Prophet forecasts
    """
    # Read Prophet forecasts from Excel files or other sources
    prophet_lag1_url = f's3://{bucket_name}/{prophet_training_prefix}/{prophet_filename}'
    all_parts_prophet_lag1 = pd.read_excel(prophet_lag1_url)
    

    # Initialize an empty dataframe to store combined lag 1 data
    all_lag1_data_prophet = pd.DataFrame()

    all_parts_prophet_lag1 = all_parts_prophet_lag1.rename(columns={'forecasted_values':'Prophet',
                                                                    'actual':'Total_qty_per_month'}).drop(['model_name', 
                                                                                                                   'accuracy_pred'], axis=1)

    all_parts_prophet_lag1 = all_parts_prophet_lag1[["period", "Prophet", "part_number", "Total_qty_per_month", "type"]]
    test_date_range = get_consecutive_months(training_end_date)
    all_parts_prophet_lag1 = all_parts_prophet_lag1[all_parts_prophet_lag1['period'].isin(test_date_range)].drop('type', axis=1)
    all_parts_prophet_lag1 = all_parts_prophet_lag1[['period', 'part_number', 'Total_qty_per_month', 'Prophet']]
    return all_parts_prophet_lag1

In [16]:
def lstm_best_fit_lag1(bucket_name, lstm_training_prefix, lstm_filename, training_end_date):
    """
    Combine LSTM forecasts from three different lags into a single formatted dataframe for lag 1.

    Args:
    - lstm_lag1_url (str): File path or URL to LSTM forecasts
    - training_end_date: end date for training data

    Returns:
    - DataFrame: Combined and formatted dataframe containing LSTM forecasts for lag 1.
    """
    # Read LSTM forecasts from Excel files or other sources
    lstm_lag1_url = f's3://{bucket_name}/{lstm_training_prefix}/{lstm_filename}'
    print(lstm_lag1_url)
    lstm_data_lag1 = pd.read_excel(lstm_lag1_url)
    
    # Convert 'period' column to string format for filtering
    lstm_data_lag1['period'] = pd.to_datetime(lstm_data_lag1['period']).astype(str)
    test_date_range = get_consecutive_months(training_end_date)

    # Filter data for specific periods (assumed to be the last three months of the forecast)
    lstm_data_lag1 = lstm_data_lag1[lstm_data_lag1['period'].isin(test_date_range)].drop('last_history', 
                                                                                         axis=1).reset_index(drop=True).rename(columns={'Forecast':'LSTM single model', 
                                                                                                                                        'Actual':'Total_qty_per_month'})
    
    
    return lstm_data_lag1

In [17]:
def lasso_best_fit_lag1(bucket_name, lasso_training_prefix, lasso_filename, training_end_date):
    """
    Combine and format Lasso regression forecasts from three different lags into a single dataframe for lag 1.

    Args:
    - lasso_lag1_url (str): File path or URL to Lasso regression forecasts

    Returns:
    - DataFrame: Combined and formatted dataframe containing Lasso regression forecasts for lag 1.
    """
    # Read Lasso regression forecasts from Excel files or other sources
    lasso_lag1_url = f's3://{bucket_name}/{lasso_training_prefix}/{lasso_filename}'
    lasso_data_lag1 = pd.read_excel(lasso_lag1_url)
    test_date_range = get_consecutive_months(training_end_date)
    lasso_data_lag1['period'] = pd.to_datetime(lasso_data_lag1['period']).astype(str)
    
    # Filter data for specific periods (assumed to be the last three months of the forecast)
    lasso_lag1_subset = lasso_data_lag1[lasso_data_lag1['period'].isin(test_date_range)].reset_index(drop=True).rename(columns={'Predicted_values_lag1':'Lasso',
                                                                                                                               'Actual_values':'Total_qty_per_month'})
    
    return lasso_lag1_subset

In [18]:
def ridge_best_fit_lag1(bucket_name, ridge_training_prefix, ridge_filename, training_end_date):
    """
    Combine and format Ridge regression forecasts from three different lags into a single dataframe for lag 1.

    Args:
    - ridge_lag1_url (str): File path or URL to Ridge regression forecasts for lag 1.
    - training_end_date: end date for training data

    Returns:
    - DataFrame: Combined and formatted dataframe containing Ridge regression forecasts for lag 1.
    """
    # Read Ridge regression forecasts from Excel files or other sources
    ridge_lag1_url = f's3://{bucket_name}/{ridge_training_prefix}/{ridge_filename}'
    ridge_data_lag1 = pd.read_excel(ridge_lag1_url)
    test_date_range = get_consecutive_months(training_end_date)
    ridge_data_lag1['period'] = pd.to_datetime(ridge_data_lag1['period']).astype(str)
    
    # Filter data for specific periods (assumed to be the last three months of the forecast)
    ridge_lag1_subset = ridge_data_lag1[ridge_data_lag1['period'].isin(test_date_range)].reset_index(drop=True).rename(columns={'Predicted_values_lag1':'Ridge',
                                                                                                                               'Actual_values':'Total_qty_per_month'})
    
    return ridge_lag1_subset

In [19]:
def xgboost_best_fit_lag1(bucket_name, xgboost_training_prefix, xgboost_filename, training_end_date):
    """
    Combine and format XGBoost regression forecasts

    Args:
    - xgboost_lag1_url (str): File path or URL to XGBoost regression forecasts for lag 1.
    - training_end_date: end date for training data

    Returns:
    - DataFrame: Combined and formatted dataframe containing XGBoost regression forecasts for lag 1.
    """
    # Read XGBoost regression forecasts from Excel files or other sources
    xgboost_lag1_url = f's3://{bucket_name}/{xgboost_training_prefix}/{xgboost_filename}'
    xgb_data_lag1 = pd.read_excel(xgboost_lag1_url)
    test_date_range = get_consecutive_months(training_end_date)
    xgb_data_lag1['period'] = pd.to_datetime(xgb_data_lag1['period']).astype(str)
    
    # Filter data for specific periods (assumed to be the last three months of the forecast)
    xgb_lag1_subset = xgb_data_lag1[xgb_data_lag1['period'].isin(test_date_range)].reset_index(drop=True).rename(columns={'Predicted_values_lag1':'XGBoost',
                                                                                                                               'Actual_values':'Total_qty_per_month'})   
    return xgb_lag1_subset

In [20]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [21]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [22]:
prophet_data = pd.read_excel(f"s3://{bucket}/{prefix_output}/{bestfit_file_prophet}")
training_end_date = (pd.to_datetime(prophet_data['period']).max() + relativedelta(months=-3)).strftime('%Y-%m-%d')

In [23]:
prophet_lag1_combined = prophet_best_fit_lag1(bucket, 
                                              prefix_output, 
                                              bestfit_file_prophet,
                                              training_end_date
                                             )

In [24]:
lstm_lag1_combined = lstm_best_fit_lag1(bucket, 
                                        prefix_output, 
                                        bestfit_file_lstm,
                                        training_end_date
                                        )

s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/model-training-output/train_lstm_forecast_1_2024-09-13.xlsx


In [25]:
# bestfit_file_lasso

In [26]:
lasso_lag1_combined = lasso_best_fit_lag1(bucket, 
                                          prefix_output, 
                                          bestfit_file_lasso,
                                          training_end_date
                                         )

In [27]:
ridge_lag1_combined = ridge_best_fit_lag1(bucket, 
                                          prefix_output, 
                                          bestfit_file_ridge,
                                          training_end_date
                                         )

In [28]:
xgboost_lag1_combined = xgboost_best_fit_lag1(bucket, 
                                              prefix_output, 
                                              bestfit_file_xgboost,
                                              training_end_date
                                             )

In [29]:
def combined_data_lag1(prophet_lag, lstm_lag, lasso_lag, ridge_lag, xgboost_lag):
    """
    Combine forecasts from Prophet, LSTM, Lasso, Ridge, and XGBoost models for lag 1,
    calculate MAPE for each model, identify the best-fitting model for each part number,
    and structure the results in a pivot table format.

    Args:
    - prophet_lag (DataFrame): Dataframe containing Prophet model forecasts for lag 1.
    - lstm_lag (DataFrame): Dataframe containing LSTM model forecasts for lag 1.
    - lasso_lag (DataFrame): Dataframe containing Lasso model forecasts for lag 1.
    - ridge_lag (DataFrame): Dataframe containing Ridge model forecasts for lag 1.
    - xgboost_lag (DataFrame): Dataframe containing XGBoost model forecasts for lag 1.

    Returns:
    - DataFrame: Pivot table containing part numbers, best-fitting model, actual values,
                forecasted values, and accuracy predictions for each period.
    """
    # Transform all part number into string format
    prophet_lag['part_number'] = prophet_lag['part_number'].astype(str)
    lstm_lag['part_number'] = lstm_lag['part_number'].astype(str)
    lasso_lag['part_number'] = lasso_lag['part_number'].astype(str)
    ridge_lag['part_number'] = ridge_lag['part_number'].astype(str)
    xgboost_lag['part_number'] = xgboost_lag['part_number'].astype(str)

    prophet_lag['period'] = pd.to_datetime(prophet_lag['period']).astype(str)
    xgboost_lag['period'] = pd.to_datetime(xgboost_lag['period']).astype(str)
    lasso_lag['period'] = pd.to_datetime(lasso_lag['period']).astype(str)
    ridge_lag['period'] = pd.to_datetime(ridge_lag['period']).astype(str)
    lstm_lag['period'] = pd.to_datetime(lstm_lag['period']).astype(str)
    
    # Merge all forecasts into a single dataframe
    fin_data = pd.merge(prophet_lag1_combined, 
                         pd.merge(xgboost_lag1_combined.drop('Total_qty_per_month', axis=1),
                                  pd.merge(lasso_lag1_combined.drop('Total_qty_per_month', axis=1),
                                           pd.merge(ridge_lag1_combined.drop('Total_qty_per_month', axis=1),
                                                    lstm_lag1_combined.drop('Total_qty_per_month', axis=1),
                                                    on=['period', 'part_number'], how='left'),
                                           on=['period', 'part_number'], how='left'),
                                  on=['period', 'part_number'], how='left'),
                         on=['period', 'part_number'], how='left')

    # Calculate MAPE for each model
    all_lag1_data_sp3_all_models_mape = fin_data.iloc[:,:3]
    for cols in fin_data.iloc[:,3:].columns:
        all_lag1_data_sp3_all_models_mape[cols] = fin_data.apply(lambda row: calculate_mape(row['Total_qty_per_month'], row[cols]), axis=1)
    
    # Prepare dataframe for storing MAPE values
    all_forecasted_data_mape_df = pd.DataFrame()

    # Iterate over model columns to calculate mean MAPE per part number
    for model_name in all_lag1_data_sp3_all_models_mape.columns[3:].tolist():
        model_mape_df = all_lag1_data_sp3_all_models_mape.groupby(['part_number'])[model_name].mean().reset_index()
        model_mape_df = model_mape_df.melt(id_vars='part_number', 
                                           var_name=model_name, 
                                           value_vars=model_name)
        model_mape_df.columns = ['part_number', 'model', 'value']
        all_forecasted_data_mape_df = pd.concat([all_forecasted_data_mape_df, model_mape_df], ignore_index=True)

    # Select the rows with maximum MAPE for each part number
    all_forecasted_data_mape_df_max = all_forecasted_data_mape_df.loc[all_forecasted_data_mape_df.groupby('part_number')['value'].idxmax()].reset_index(drop=True)

    # Create final dataframe to store best-fit forecasts and accuracy predictions
    final_df_best_fit = pd.DataFrame()

    # Iterate over part numbers to select best-fitting model and populate final dataframe
    for pnt in all_forecasted_data_mape_df_max['part_number'].unique():
        best_model_name = all_forecasted_data_mape_df_max[all_forecasted_data_mape_df_max['part_number']==pnt]['model'].values[0]
        forecasted_data_subset_pnt = fin_data.loc[fin_data['part_number']==pnt, 
                                                                ['part_number', 'period', 
                                                                 'Total_qty_per_month', best_model_name]]
        forecasted_data_subset_pnt['model_name'] = best_model_name
        forecasted_data_subset_pnt.rename(columns={best_model_name:'forecasted_values'}, inplace=True)
        final_df_best_fit = pd.concat([final_df_best_fit, forecasted_data_subset_pnt], ignore_index=True)

    # Transform dates into string
    final_df_best_fit['period'] = final_df_best_fit['period'].astype(str)

    # Calculate accuracy prediction based on actual and forecasted values
    final_df_best_fit = final_df_best_fit.assign(accuracy_pred = 
                                                 lambda x: np.where(x['Total_qty_per_month']!=0,
                                                                    (1-(abs(x['forecasted_values'] - x['Total_qty_per_month'])/x['Total_qty_per_month'])),
                                                                    0))

    # Rename columns for clarity
    final_df_best_fit.rename(columns={'Total_qty_per_month': 'actual', 'forecasted_values': 'forecasted'}, inplace=True)

    # Create pivot table to summarize results
    pivot_df = final_df_best_fit.rename(columns={'Total_qty_per_month':'actual', 
                                                 'forecasted_values':'forecasted'}).pivot_table(index=['part_number', 'model_name'], 
                                                                                                columns='period', 
                                                                                                values=['actual', 'forecasted', 'accuracy_pred'], 
                                                                                                sort=False)
    # Flatten the column names
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
    # Sort the columns
    pivot_df = pivot_df.sort_index(axis=1).reset_index()

    return pivot_df

In [30]:
final_data_compared_bestfit_1 = combined_data_lag1(prophet_lag1_combined, lstm_lag1_combined, lasso_lag1_combined, ridge_lag1_combined, xgboost_lag1_combined)

In [31]:
final_data_compared_bestfit_1.shape

(6525, 11)

In [32]:
def final_data_bestfit_comparison(final_data_compared_bestfit):
    # Extract the static columns and the dynamic date columns
    static_columns = ['part_number', 'model_name']
    date_columns = [col for col in final_data_compared_bestfit.columns if col not in static_columns]

    # Function to extract date and type
    def extract_date_and_type(col_name):
        parts = col_name.split('_')
        date = parts[0]
        col_type = '_'.join(parts[1:])
        return date, col_type

    # Sort the date columns based on date and type
    sorted_date_columns = sorted(date_columns, key=lambda x: (extract_date_and_type(x)[0], ['actual', 'forecasted', 'accuracy_pred'].index(extract_date_and_type(x)[1])))

    # Combine static columns with sorted date columns
    sorted_columns = static_columns + sorted_date_columns

    # Reorder the DataFrame columns
    final_data_compared_bestfit = final_data_compared_bestfit[sorted_columns]

    actual_columns = [col for col in date_columns if 'actual' in col]
    forecasted_columns = [col for col in date_columns if 'forecasted' in col]
    date_columns_sorted = [col for col in final_data_compared_bestfit.columns if col not in static_columns]
    final_data_compared_bestfit['Total_Actual_Demand'] = final_data_compared_bestfit[actual_columns].sum(axis=1)
    final_data_compared_bestfit['Total_Forecasted_Demand'] = final_data_compared_bestfit[forecasted_columns].sum(axis=1)
    final_data_compared_bestfit['Closeness_to_accuracy'] = final_data_compared_bestfit.apply(lambda row: 
                                                             calculate_mape(row['Total_Actual_Demand'], 
                                                                row['Total_Forecasted_Demand']), axis=1)
    final_data_compared_bestfit['Confidence_interval'] = '#N/A'
    final_data_compared_bestfit.loc[final_data_compared_bestfit['model_name']=="Prophet", 'Confidence_interval'] = 0.8
    
    def create_bucket(df, column_name, new_col_name):
        # Define the conditions dynamically
        conditions = [
            (df[column_name] < 0.3),
            (df[column_name] >= 0.3) & (df[column_name] < 0.5),
            (df[column_name] >= 0.5) & (df[column_name] < 0.6),
            (df[column_name] >= 0.6) & (df[column_name] < 0.79),
            (df[column_name] >= 0.8)
        ]

        # Define the choices
        choices = [
            "< 30",
            "30-50",
            "50-60",
            "60-79",
            "> 80"
        ]

        # Use numpy.select to apply the conditions and choices dynamically
        df['Revised_Bucket'] = np.select(conditions, choices, default="")

        # Display the DataFrame
        return df

    create_bucket(final_data_compared_bestfit, 'Closeness_to_accuracy', 'Revised_Bucket')

    final_data_compared_bestfit['Avg_residual'] = (final_data_compared_bestfit['Total_Actual_Demand'] - final_data_compared_bestfit['Total_Forecasted_Demand'])/3
    final_data_compared_bestfit = final_data_compared_bestfit.assign(Variation = lambda x: np.where(np.logical_and(x['Avg_residual'] > -10, x['Avg_residual'] < 10), 1, 0))
    
    def create_threshold(acc, var):
        if acc >= 0.8:
            return True
        elif acc<0.8 and var==1:
            return True
        else:
            return False

    final_data_compared_bestfit['Threshold'] = final_data_compared_bestfit.apply(lambda x:create_threshold(x['Closeness_to_accuracy'], x['Variation']), axis=1)
    
    # Melt the DataFrame
    date_cols = date_columns_sorted[:]
    df_melted = pd.melt(final_data_compared_bestfit, id_vars=['part_number', 'model_name', 'Total_Actual_Demand', 'Total_Forecasted_Demand', 
                                     'Closeness_to_accuracy', 'Confidence_interval', 'Revised_Bucket', 'Avg_residual', 
                                     'Variation', 'Threshold'],
                        value_vars=date_cols, 
                        var_name='date_measure', 
                        value_name='value')

    # Extract date and measure type
    df_melted[['date', 'measure']] = df_melted['date_measure'].str.extract(r'(\d{4}-\d{2}-\d{2})_(\w+)')
    df_melted.drop('date_measure', axis=1, inplace=True)

    # Pivot the DataFrame
    df_pivoted = df_melted.pivot_table(index=['part_number', 'model_name', 'date'], 
                                       columns='measure', 
                                       values='value', 
                                       aggfunc='first').reset_index()

    # Merge with static columns
    static_cols = final_data_compared_bestfit.drop(columns=date_cols).drop_duplicates(subset=['part_number', 'model_name'])
    result = pd.merge(df_pivoted, static_cols, on=['part_number', 'model_name'])

    # Rename columns for clarity
    result.columns.name = None
    result = result.rename(columns={
        'actual': 'Actual',
        'forecasted': 'Forecasted',
        'accuracy_pred': 'Accuracy_Pred'
    })

    result_subset_1 = result.drop(['Total_Actual_Demand', 'Total_Forecasted_Demand', 'Avg_residual'], axis=1)
    return result_subset_1

In [57]:
bucket_new = config.get_value('bucket_new')
prefix_bestfit_file = config.get_value('prefix_bestfit_file')
file_name_bestfit = f'{config.get_value("file_name_bestfit")}{file_date_training}.xlsx'
file_path_bestfit = f's3://{bucket_new}/{prefix_bestfit_file}/{file_name_bestfit}'

final_data_compared_bestfit_last = final_data_bestfit_comparison(final_data_compared_bestfit_1)
final_data_compared_bestfit_last.to_excel(file_path_bestfit, index=False)

In [56]:
file_path_bestfit

's3://s3-dx-tdem-ie-nonprod-curated/curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_2024-09-13.xlsx'

In [35]:
final_data_compared_bestfit_last.shape

(19575, 11)

In [55]:
final_data_compared_bestfit_last

,part_number,model_name,date,Accuracy_Pred,Actual,Forecasted,Closeness_to_accuracy,Confidence_interval,Revised_Bucket,Variation,Threshold
0,04000081B0,Prophet,2024-04-01,0.897352,8.0,7.178818,0.753194,0.8,60-79,1,True
1,04000081B0,Prophet,2024-05-01,-0.947245,2.0,5.894491,0.753194,0.8,60-79,1,True
2,04000081B0,Prophet,2024-06-01,0.954943,3.0,3.135172,0.753194,0.8,60-79,1,True
3,0411106101,Prophet,2024-04-01,0.000000,0.0,16.051309,-3.323518,0.8,< 30,1,True
4,0411106101,Prophet,2024-05-01,0.000000,0.0,5.613132,-3.323518,0.8,< 30,1,True
...,...,...,...,...,...,...,...,...,...,...,...
19570,PZ08100005,Ridge,2024-05-01,0.988924,156.0,154.272150,0.953283,#N/A,> 80,1,True
19571,PZ08100005,Ridge,2024-06-01,0.851722,153.0,130.313446,0.953283,#N/A,> 80,1,True
19572,PZT0038023,Prophet,2024-04-01,0.747158,320.0,239.090712,0.933336,0.8,> 80,0,True
19573,PZT0038023,Prophet,2024-05-01,0.927355,305.0,327.156868,0.933336,0.8,> 80,0,True


# power BI process starts

In [36]:
bucket_curated = config.get_value('bucket_new')
prefix_gtmaster = config.get_value('prefix_gtmaster')
prefix_proc = config.get_value('prefix_proc')
prefix_pic_master_path = config.get_value("prefix_pic_master_path")
product_master_path_prefix = config.get_value('product_master_path_prefix')

In [37]:
# def dashboard_builder(final_data_compared_bestfit_last_1):    
#     part_master=pd.read_excel(part_master_path_1)
#     part_master['ICC_seg'] = part_master['ICC'].apply(lambda x:x[0])
#     part_master['PART_NO'] = part_master['PART_NO'].astype(str)
#     result_subset=final_data_compared_bestfit_last_1[:]
#     result_subset['part_number'] = result_subset['part_number'].astype(str)
#     result_subset = result_subset.merge(part_master[['PART_NO', 'ICC_seg']].drop_duplicates(), 
#                                         left_on='part_number', 
#                                         right_on='PART_NO', 
#                                         how='left').drop('PART_NO', 
#                                                          axis=1)
#     result_subset = result_subset[['part_number', 'model_name', 'ICC_seg','Actual',
#            'Forecasted','Accuracy_Pred', 'Closeness_to_accuracy', 'Confidence_interval',
#            'Revised_Bucket', 'Variation', 'Threshold', 'date']]
#     result_subset = result_subset.rename(columns = {'part_number': 'Part Number', 'model_name': 'Model Name', 
#                                                     'Accuracy_Pred': 'New Model Accuracy', 'Closeness_to_accuracy':'Total Accuracy',
#                                                     'date': 'Date', 'ICC_seg':'ICC'
#                                                    })
#     result_subset['Lag'] = 'Lag1'
#     result_subset['Part Number'] = result_subset['Part Number'].astype(str)

#     # GT Part Master data
#     # Specify the column names and their types
#     dtype = {
#         'PROCUREMENT_OPERATOR_ID': str  # 'Part Number' column will be read as strings
#     }
#     data_gt_master = pd.read_csv(data_gt_master_path_1)

#     # PIC master data
#     dtype_pic = {
#         'ID': str  # 'Part Number' column will be read as strings
#     }
#     data_pic_master = pd.read_csv(data_pic_master_path_1)

#     # Supplier master
#     data_supp_master = pd.read_csv(data_supp_master_path_1)
#     data_gt_master['PART_NO'] = data_gt_master['PART_NO'].astype(str)
#     all_data_gt_concat = result_subset.merge(data_gt_master, left_on='Part Number', right_on='PART_NO', how='left')
#     all_data_gt_concat_supp_pic = pd.merge(all_data_gt_concat,data_supp_master[['Vendor CD', 'SUPPLIER_NAME']], 
#                                            left_on='Vendor Code', 
#                                            right_on='Vendor CD', 
#                                            how='left').merge(data_pic_master, left_on='PROCUREMENT_OPERATOR_ID', right_on='ID', how='left')
#     all_data_gt_concat_supp_pic_subset = all_data_gt_concat_supp_pic[['Part Number', 'Model Name', 'ICC', 'Actual', 'Forecasted',
#            'New Model Accuracy', 'Threshold','Date', 'Lag',
#            'PART_NAME', 'Vendor Code','SUPPLIER_NAME', 'PIC',
#            'PIC (Abb)']]
#     all_data_gt_concat_supp_pic_subset_pm_data = pd.merge(all_data_gt_concat_supp_pic_subset, 
#                                                           part_master[['PART_NO', 'IMPORT_CD', 'PRODUCT_GROUP']],
#                                                           left_on='Part Number', right_on = 'PART_NO', how='left'
#                                                          )
#     all_data_gt_concat_supp_pic_subset_pm_data = all_data_gt_concat_supp_pic_subset_pm_data.drop('PART_NO', axis=1)

#     all_data_sorted = all_data_gt_concat_supp_pic_subset_pm_data[['Part Number', 'Model Name', 'ICC','Date', 'Actual', 'Forecasted',
#                                                                   'New Model Accuracy', 'PRODUCT_GROUP', 'IMPORT_CD', 'Lag', 'SUPPLIER_NAME', 'PIC', 'PIC (Abb)', 'Threshold']]

#     all_data_sorted['Year'] = pd.to_datetime(all_data_sorted['Date']).dt.year
#     all_data_sorted['Month Name'] = pd.to_datetime(all_data_sorted['Date']).dt.month_name()
#     all_data_sorted['Month'] = pd.to_datetime(all_data_sorted['Date']).dt.month
#     all_data_sorted['sort'] = pd.to_datetime(all_data_sorted['Date']).dt.year.astype(str) + pd.to_datetime(all_data_sorted['Date']).dt.month.astype(str)
#     all_data_sorted['months'] = pd.to_datetime(all_data_sorted['Date']).dt.strftime('%b').astype(str)+pd.to_datetime(all_data_sorted['Date']).dt.strftime('%y').astype(str)

#     col_names_list = """Part Number
#     Model Name
#     ICC
#     Date
#     Actual
#     Forecasted
#     Old Model Accuracy
#     New Model Accuracy
#     Product Group
#     Source Type
#     Lag
#     Supplier Name
#     Person in Charge
#     PIC (Abb)
#     Threshold
#     Year
#     Month Name
#     Month
#     sort
#     months""".split('\n')
#     col_names_list = list(map(lambda x:x.strip(), col_names_list))
#     all_data_sorted['Old Model Accuracy']=np.nan
#     all_data_sorted = all_data_sorted[['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted','Old Model Accuracy',
#            'New Model Accuracy', 'PRODUCT_GROUP', 'IMPORT_CD', 'Lag',
#            'SUPPLIER_NAME', 'PIC', 'PIC (Abb)', 'Threshold','Year', 'Month Name', 'Month',
#            'sort', 'months']]
#     all_data_sorted.columns = col_names_list
#     all_data_sorted['Forecast_flag'] = 'Historic'
#     return all_data_sorted

In [38]:
# Function to read a single Parquet file
def read_parquet_file(file):
    return pd.read_parquet(f's3://{file}', engine='pyarrow')

def gt_parquet_read(bucket_curated_name, latest_gt_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{gt_master_parquet}/*/*/*/*.parquet')
    parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{latest_gt_master_parquet}*.parquet') ###UPDATED

    df_master_gt_part_master = pd.DataFrame()
    for file in parquet_files_gt_partm:
        df_gt = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_gt_part_master = pd.concat([df_master_gt_part_master,df_gt],ignore_index=True)
    df_master_gt_part_master_subset = df_master_gt_part_master.sort_values('sales_start_dt', ascending=False)
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop_duplicates(subset=['part_no'], keep='first')
    df_master_gt_part_master_subset = df_master_gt_part_master_subset[["part_no", "country_origin_cd"]].drop_duplicates()
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.assign(IMPORT_CD = 
                                                                             lambda x: np.where(x['country_origin_cd']=='TH', 
                                                                                                'LSP', 
                                                                                                np.where(x['country_origin_cd']=='JP', 
                                                                                                         'JSP', 'MSP')))
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop('country_origin_cd', axis=1)
    df_master_gt_part_master_subset.columns = [col.upper() for col in df_master_gt_part_master_subset.columns]
    return df_master_gt_part_master_subset

def proc_parquet_read(bucket_curated_name, latest_proc_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{proc_master_parquet}/*/*/*/*.parquet')
    parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{latest_proc_master_parquet}*.parquet') ###UPDATED

    df_master_proc_master = pd.DataFrame()

    for file in parquet_files_proc:
    #     print(file)
        df_proc = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_proc_master = pd.concat([df_master_proc_master,df_proc],ignore_index=True)
    df_master_proc_master = df_master_proc_master.sort_values(['start_dt'], ascending=[False])
    df_master_proc_master = df_master_proc_master.drop_duplicates(subset='part_no', keep='first')
    df_master_proc_master_subset = df_master_proc_master[['part_no', 'procurement_div_cd', 'procurement_operator_id', 'inventory_ctrl_class']].drop_duplicates()
    df_master_proc_master_subset['procurement_div_cd'] = df_master_proc_master_subset['procurement_div_cd'].apply(lambda x:x[-3:])
    df_master_proc_master_subset = df_master_proc_master_subset.rename(columns={'procurement_div_cd':'Vendor Code',
                                                                                'part_no':'PART_NO',
                                                                                'procurement_operator_id':'PROCUREMENT_OPERATOR_ID',
                                                                                'inventory_ctrl_class': 'ICC_seg'
                                                                       })
    df_master_proc_master_subset['ICC_seg'] = df_master_proc_master_subset['ICC_seg'].apply(lambda x: x[0] if pd.notna(x) and len(x) > 1 else x)
    return df_master_proc_master_subset

def pic_parquet_read(bucket_curated_name, latest_pic_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
    parquet_files_pic_master = fs.glob(f's3://{bucket_curated_name}/{latest_pic_master_parquet}*.parquet')

    df_master_pic = pd.DataFrame()

    for file in parquet_files_pic_master:
    #     print(file)
        df_pic = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_pic = pd.concat([df_master_pic,df_pic],ignore_index=True)
    
    df_master_pic_subset = df_master_pic[['id', 'pic', 'pic_abb']].drop_duplicates().reset_index(drop=True)
    df_master_pic_subset.columns = [col.upper() for col in df_master_pic_subset.columns]

    return df_master_pic_subset

def prod_master_read(bucket_curated_name, pg_master_path):
    pg_master_filepath = "s3://"+fs.glob(f's3://{bucket_curated_name}/{pg_master_path}/*.csv')[0]
    pg_master_df = read_file(pg_master_filepath)
    pg_master_df = pg_master_df[['PART_NO', 'PRODUCT_GROUP']].drop_duplicates()
    return pg_master_df

In [39]:
datetime.now()

datetime.datetime(2024, 10, 22, 9, 22, 14, 338884)

In [40]:
# bucket_cur = bucket_curated
# gt_parquet_path = prefix_gtmaster
# proc_parquet_path = prefix_proc
# pic_parquet_path = prefix_pic_master_path
# prod_path = product_master_path_prefix
# gt_data = gt_parquet_read(bucket_cur, gt_parquet_path)
# proc_data = proc_parquet_read(bucket_cur, proc_parquet_path)
# pic_data = pic_parquet_read(bucket_cur, pic_parquet_path)
# pg_data = prod_master_read(bucket_cur, prod_path)

In [41]:
def get_lastest_partition_path(bucket_name, prefix):
    if prefix[-1] != "/": prefix += "/"
    
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    year_month_day = {}

    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
            for obj in page.get('Contents', []):
                key = obj['Key']
                key = key[len(prefix):]
                parts = key.split('/')
#                 print(parts)
                if len(parts) >= 3:
                    if "=" in parts[0] and "=" in parts[1]:
                        year = parts[0].split('=')[-1]
                        month = parts[1].split('=')[-1]
                        if "=" in parts[2]:
                            day = parts[2].split('=')[-1]
                        else: day = ""
                            
                        if len(day) > 2: continue

                        if year not in year_month_day:
                            year_month_day[year] = {}
                        if month not in year_month_day[year]:
                            year_month_day[year][month] = set()
                        year_month_day[year][month].add(day)

#     print(year_month_day.keys())
    
    most_recent_year = max(year_month_day.keys(), key=int)
    most_recent_month = max(year_month_day[most_recent_year].keys(), key=int)
    if year_month_day[most_recent_year][most_recent_month] != {""}:
        most_recent_day = max(year_month_day[most_recent_year][most_recent_month], key=int)
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/day={most_recent_day}/"
    else:
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/"
        
    return final_path

In [42]:
def create_final_master_file(bucket_cur, gt_parquet_path, proc_parquet_path, pic_parquet_path, prod_path):
    prefix_gtmaster = get_lastest_partition_path(bucket_cur, gt_parquet_path)
    gt_data = gt_parquet_read(bucket_cur, prefix_gtmaster)
    prefix_procmaster = get_lastest_partition_path(bucket_cur, proc_parquet_path)
    proc_data = proc_parquet_read(bucket_cur, prefix_procmaster)
    prefix_picmaster = get_lastest_partition_path(bucket_cur, pic_parquet_path)
    pic_data = pic_parquet_read(bucket_cur, prefix_picmaster)
    
    pg_data = prod_master_read(bucket_cur, prod_path)
    
    pic_data['ID'] = pic_data['ID'].astype(str)
    gt_data['PART_NO'] = gt_data['PART_NO'].astype(str)
    proc_data['PART_NO'] = proc_data['PART_NO'].astype(str)
    pg_data['PART_NO'] = pg_data['PART_NO'].astype(str)
    
    final_master_file_data = pd.merge(pd.merge(pd.merge(gt_data, proc_data, on='PART_NO', how='left'), 
                                 pic_data, left_on='PROCUREMENT_OPERATOR_ID', 
                                 right_on='ID', how='left').drop('ID', axis=1), 
                        pg_data, on='PART_NO', how='left')
    
    return final_master_file_data

In [43]:
full_master_details = create_final_master_file(bucket_curated, prefix_gtmaster, prefix_proc, prefix_pic_master_path, product_master_path_prefix)

In [44]:
datetime.now()

datetime.datetime(2024, 10, 22, 9, 22, 32, 369684)

In [45]:
full_master_details.shape, full_master_details['PART_NO'].nunique()

((631326, 8), 631326)

In [46]:
def dashboard_builder(final_data_compared_bestfit_last_1, final_master_data):    
    data_train_bestfit_set = final_data_compared_bestfit_last_1[['part_number', 'model_name','Actual',
                                         'Forecasted','Accuracy_Pred', 'Closeness_to_accuracy', 
                                         'Confidence_interval','Revised_Bucket', 'Variation', 
                                         'Threshold', 'date']]

    data_train_bestfit_set = data_train_bestfit_set.rename(columns = {'part_number': 'Part Number', 'model_name': 'Model Name', 
                                                        'Accuracy_Pred': 'New Model Accuracy', 'Closeness_to_accuracy':'Total Accuracy',
                                                        'date': 'Date'
                                                       })
    data_train_bestfit_set['Lag'] = 'Lag1'
    
    data_train_bestfit_set_master = data_train_bestfit_set.merge(final_master_data, 
                                                             left_on='Part Number', 
                                                             right_on='PART_NO', 
                                                             how='left').drop(['PART_NO', 'PROCUREMENT_OPERATOR_ID'], 
                                                                              axis=1).rename(columns={'ICC_seg':'ICC',
                                                                                                      'Vendor Code':'SUPPLIER_NAME',
                                                                                                      'PIC_ABB':'PIC (Abb)'
                                                                                                     })
    all_data_train_sorted = data_train_bestfit_set_master[['Part Number', 'Model Name', 'ICC',
                                                           'Date', 'Actual', 'Forecasted','New Model Accuracy', 
                                                           'PRODUCT_GROUP', 'IMPORT_CD', 'Lag', 'SUPPLIER_NAME', 
                                                           'PIC', 'PIC (Abb)', 'Threshold']]
    
    all_data_train_sorted['Year'] = pd.to_datetime(all_data_train_sorted['Date']).dt.year
    all_data_train_sorted['Month Name'] = pd.to_datetime(all_data_train_sorted['Date']).dt.month_name()
    all_data_train_sorted['Month'] = pd.to_datetime(all_data_train_sorted['Date']).dt.month
    all_data_train_sorted['sort'] = pd.to_datetime(all_data_train_sorted['Date']).dt.year.astype(str) + pd.to_datetime(all_data_train_sorted['Date']).dt.month.astype(str)
    all_data_train_sorted['months'] = pd.to_datetime(all_data_train_sorted['Date']).dt.strftime('%b').astype(str)+pd.to_datetime(all_data_train_sorted['Date']).dt.strftime('%y').astype(str)

    col_names_list = """Part Number
    Model Name
    ICC
    Date
    Actual
    Forecasted
    Old Model Accuracy
    New Model Accuracy
    Product Group
    Source Type
    Lag
    Supplier Name
    Person in Charge
    PIC (Abb)
    Threshold
    Year
    Month Name
    Month
    sort
    months""".split('\n')
    col_names_list = list(map(lambda x:x.strip(), col_names_list))
    all_data_train_sorted['Old Model Accuracy']=np.nan
    all_data_train_sorted = all_data_train_sorted[['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted','Old Model Accuracy',
           'New Model Accuracy', 'PRODUCT_GROUP', 'IMPORT_CD', 'Lag',
           'SUPPLIER_NAME', 'PIC', 'PIC (Abb)', 'Threshold','Year', 'Month Name', 'Month',
           'sort', 'months']]
    all_data_train_sorted.columns = col_names_list
    all_data_train_sorted['Forecast_flag'] = 'Historic'
    
    return all_data_train_sorted

In [47]:
final_data_sorted=dashboard_builder(final_data_compared_bestfit_last, full_master_details)

In [49]:
f'{config.get_value("file_name_bestfit_powerbi")}{file_date_training}.xlsx'

'bestfit_selected_models_historic_months_powerbi_2024-09-13.xlsx'

In [48]:
file_name_bestfit_powerbi = f'{config.get_value("file_name_bestfit_powerbi")}{file_date_training}.xlsx'
file_path_bestfit_powerbi = f's3://{bucket_new}/{prefix_bestfit_file}/{file_name_bestfit_powerbi}'
final_data_sorted.to_excel(file_path_bestfit_powerbi, index=False)

In [54]:
file_path_bestfit_powerbi

's3://s3-dx-tdem-ie-nonprod-curated/curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_powerbi_2024-09-13.xlsx'

In [49]:
final_data_sorted.columns

Index(['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted',
       'Old Model Accuracy', 'New Model Accuracy', 'Product Group',
       'Source Type', 'Lag', 'Supplier Name', 'Person in Charge', 'PIC (Abb)',
       'Threshold', 'Year', 'Month Name', 'Month', 'sort', 'months',
       'Forecast_flag'],
      dtype='object')

In [50]:
prefix_bestfit_file_stop_instance = config.get_value("prefix_bestfit_file_stop_instance")

In [51]:
config.get_value("prefix_bestfit_file_stop_instance")

'curated-bo/curated-bo-demand-forecast/bestfit-selected-models/training-final-output'

In [52]:
end_time = datetime.now()
print("Total time taken for Bestfit selection::", (end_time-start_time).seconds/60, " minutes")

Total time taken for Bestfit selection:: 4.116666666666666  minutes


In [53]:
file_path_bestfit_powerbi_stop_instance_folder = f's3://{bucket_new}/{prefix_bestfit_file_stop_instance}/{config.get_value("file_name_bestfit_powerbi")[:-1]}.xlsx'
final_data_sorted.to_excel(file_path_bestfit_powerbi_stop_instance_folder, index=False)
file_path_bestfit_powerbi_stop_instance_folder

's3://s3-dx-tdem-ie-nonprod-curated/curated-bo/curated-bo-demand-forecast/bestfit-selected-models/training-final-output/bestfit_selected_models_historic_months_powerbi.xlsx'

In [ ]:
final_data_sorted